# Machine Learning Nanodegree


## Project: **Stock Price Estimator** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2
##Needed to do this once
##import imageio
##imageio.plugins.ffmpeg.download()

%matplotlib inline

## Read in an Image

In [2]:
df = pd.read_csv('./AVGO.csv')

In [3]:
df.head(8)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-12-27,182.110001,183.699997,182.050003,182.309998,178.796310,828300
1,2016-12-28,182.639999,183.000000,179.839996,180.190002,176.717194,1403900
2,2016-12-29,179.699997,180.669998,178.630005,179.869995,176.403351,1367100
3,2016-12-30,180.369995,180.899994,176.020004,176.770004,173.363113,2542100
4,2017-01-03,178.289993,180.899994,176.539993,178.339996,174.902847,3209800
5,2017-01-04,178.259995,179.000000,176.679993,177.070007,173.657318,2396500
6,2017-01-05,176.330002,177.000000,173.880005,174.279999,170.921097,2486100
7,2017-01-06,174.210007,176.899994,173.309998,176.589996,173.186569,2314200


In [5]:
df['Open'][0:7]

0    182.110001
1    182.639999
2    179.699997
3    180.369995
4    178.289993
5    178.259995
6    176.330002
Name: Open, dtype: float64

In [6]:
avgo_first_seven_days= df[0:7].copy(deep = True)

In [7]:
avgo_first_seven_days

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-12-27,182.110001,183.699997,182.050003,182.309998,178.796310,828300
1,2016-12-28,182.639999,183.000000,179.839996,180.190002,176.717194,1403900
2,2016-12-29,179.699997,180.669998,178.630005,179.869995,176.403351,1367100
3,2016-12-30,180.369995,180.899994,176.020004,176.770004,173.363113,2542100
4,2017-01-03,178.289993,180.899994,176.539993,178.339996,174.902847,3209800
5,2017-01-04,178.259995,179.000000,176.679993,177.070007,173.657318,2396500
6,2017-01-05,176.330002,177.000000,173.880005,174.279999,170.921097,2486100


In [8]:
avgo = df[7:].copy(deep=True)

In [9]:
avgo['High - Close'] = avgo['High'] - avgo['Low']

In [10]:
avgo.head(8)


,Date,Open,High,Low,Close,Adj Close,Volume,High - Close
7,2017-01-06,174.210007,176.899994,173.309998,176.589996,173.186569,2314200,3.589996
8,2017-01-09,176.649994,178.300003,176.639999,176.970001,173.559265,2026500,1.660004
9,2017-01-10,177.490005,180.740005,176.899994,180.570007,177.089890,3018100,3.840011
10,2017-01-11,180.050003,180.470001,177.759995,179.419998,175.962036,1878700,2.710006
11,2017-01-12,178.089996,179.100006,175.399994,178.860001,175.412827,2313700,3.700012
12,2017-01-13,178.899994,180.779999,178.389999,180.449997,176.972168,2090200,2.390000
13,2017-01-17,179.479996,179.630005,177.559998,179.110001,175.658020,2628500,2.070007
14,2017-01-18,180.360001,185.070007,180.000000,185.020004,181.454117,3564200,5.070007


In [11]:
avgo.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,High - Close
246,2017-12-18,264.079987,266.510010,262.589996,264.390015,264.390015,2978600,3.920014
247,2017-12-19,263.299988,265.290009,261.269989,263.700012,263.700012,1909400,4.020020
248,2017-12-20,266.140015,266.350006,262.779999,265.640015,265.640015,1984500,3.570007
249,2017-12-21,265.950012,266.250000,261.070007,261.529999,261.529999,1775800,5.179993
250,2017-12-22,261.820007,263.369995,258.700012,262.350006,262.350006,1453000,4.669983


In [12]:
day1 = df['Adj Close'][6:-1].copy(deep=True)
day2 = df['Adj Close'][5:-2].copy(deep=True)
day3 = df['Adj Close'][4:-3].copy(deep=True)
day4 = df['Adj Close'][3:-4].copy(deep=True)
day5 = df['Adj Close'][2:-5].copy(deep=True)
day6 = df['Adj Close'][1:-6].copy(deep=True)
day7 = df['Adj Close'][0:-7].copy(deep=True)

In [13]:
day1.head()

6     170.921097
7     173.186569
8     173.559265
9     177.089890
10    175.962036
Name: Adj Close, dtype: float64

In [14]:
d1 = pd.DataFrame(data = day1)
d2 = pd.DataFrame(data = day2)
d3 = pd.DataFrame(data = day3)
d4 = pd.DataFrame(data = day4)
d5 = pd.DataFrame(data = day5)
d6 = pd.DataFrame(data = day6)
d7 = pd.DataFrame(data = day7)

In [18]:
d1 = d1.rename(columns={'Adj Close': 'Day 1'})
d1 = d1.reset_index()
d2 = d2.rename(columns={'Adj Close': 'Day 2'})
d2 = d2.reset_index()
d3 = d3.rename(columns={'Adj Close': 'Day 3'})
d3 = d3.reset_index()
d4 = d4.rename(columns={'Adj Close': 'Day 4'})
d4 = d4.reset_index()
d5 = d5.rename(columns={'Adj Close': 'Day 5'})
d5 = d5.reset_index()
d6 = d6.rename(columns={'Adj Close': 'Day 6'})
d6 = d6.reset_index()
d7 = d7.rename(columns={'Adj Close': 'Day 7'})
d7 = d7.reset_index()

In [19]:
del d1['index']
d1.head()

,Day 1
0,170.921097
1,173.186569
2,173.559265
3,177.089890
4,175.962036


In [20]:
del d2['index']
del d3['index']
del d4['index']
del d5['index']
del d6['index']
del d7['index']

In [21]:
avgo = avgo.reset_index()
del avgo['index']
avgo.head()

,Date,Open,High,Low,Close,Adj Close,Volume,High - Close
0,2017-01-06,174.210007,176.899994,173.309998,176.589996,173.186569,2314200,3.589996
1,2017-01-09,176.649994,178.300003,176.639999,176.970001,173.559265,2026500,1.660004
2,2017-01-10,177.490005,180.740005,176.899994,180.570007,177.089890,3018100,3.840011
3,2017-01-11,180.050003,180.470001,177.759995,179.419998,175.962036,1878700,2.710006
4,2017-01-12,178.089996,179.100006,175.399994,178.860001,175.412827,2313700,3.700012


In [22]:
result = pd.concat([avgo,d1,d2,d3,d4,d5,d6,d7], axis = 1, join='inner')
#result = pd.concat([avgo,d1], axis = 1, join='inner')

In [24]:
result.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,High - Close,Day 1,Day 2,Day 3,Day 4,Day 7,Day 6,Day 7
0,2017-01-06,174.210007,176.899994,173.309998,176.589996,173.186569,2314200,3.589996,170.921097,173.657318,174.902847,173.363113,178.796310,176.717194,178.796310
1,2017-01-09,176.649994,178.300003,176.639999,176.970001,173.559265,2026500,1.660004,173.186569,170.921097,173.657318,174.902847,176.717194,176.403351,176.717194
2,2017-01-10,177.490005,180.740005,176.899994,180.570007,177.089890,3018100,3.840011,173.559265,173.186569,170.921097,173.657318,176.403351,173.363113,176.403351
3,2017-01-11,180.050003,180.470001,177.759995,179.419998,175.962036,1878700,2.710006,177.089890,173.559265,173.186569,170.921097,173.363113,174.902847,173.363113
4,2017-01-12,178.089996,179.100006,175.399994,178.860001,175.412827,2313700,3.700012,175.962036,177.089890,173.559265,173.186569,174.902847,173.657318,174.902847
5,2017-01-13,178.899994,180.779999,178.389999,180.449997,176.972168,2090200,2.390000,175.412827,175.962036,177.089890,173.559265,173.657318,170.921097,173.657318
6,2017-01-17,179.479996,179.630005,177.559998,179.110001,175.658020,2628500,2.070007,176.972168,175.412827,175.962036,177.089890,170.921097,173.186569,170.921097
7,2017-01-18,180.360001,185.070007,180.000000,185.020004,181.454117,3564200,5.070007,175.658020,176.972168,175.412827,175.962036,173.186569,173.559265,173.186569
8,2017-01-19,185.100006,186.880005,184.679993,185.559998,181.983688,4571800,2.200012,181.454117,175.658020,176.972168,175.412827,173.559265,177.089890,173.559265
9,2017-01-20,188.600006,191.539993,188.100006,191.080002,187.397308,5762300,3.439987,181.983688,181.454117,175.658020,176.972168,177.089890,175.962036,177.089890
